In [8]:
import torch
import torch.nn as nn
import math

In [2]:
# --- 1. Define Our Labels ---
LABEL_MAP = {
    "Negative": 0,
    "Positive": 1
}

# --- 2. Create the Dataset ---
# We MUST add the [CLS] token to the beginning of every sentence.
# This is the special token our Transformer will use to "summarize"
# the whole sentence.
data = [
    # Positive Examples
    ("[CLS] I loved every second of it", LABEL_MAP["Positive"]),
    ("[CLS] a truly amazing film", LABEL_MAP["Positive"]),
    ("[CLS] this movie was fantastic", LABEL_MAP["Positive"]),
    ("[CLS] the acting was superb", LABEL_MAP["Positive"]),
    ("[CLS] 10 out of 10 would recommend", LABEL_MAP["Positive"]),
    ("[CLS] a masterpiece of cinema", LABEL_MAP["Positive"]),
    ("[CLS] I was on the edge of my seat", LABEL_MAP["Positive"]),
    ("[CLS] a beautiful and touching story", LABEL_MAP["Positive"]),
    ("[CLS] an absolutely breathtaking masterpiece", LABEL_MAP["Positive"]),
    ("[CLS] the acting was top notch and emotionally gripping", LABEL_MAP["Positive"]),
    ("[CLS] beautifully directed with stunning visuals", LABEL_MAP["Positive"]),
    ("[CLS] this movie touched my heart in every way", LABEL_MAP["Positive"]),
    ("[CLS] a remarkable story told with passion and depth", LABEL_MAP["Positive"]),
    ("[CLS] easily one of the best films of the year", LABEL_MAP["Positive"]),
    ("[CLS] the chemistry between the leads was magical", LABEL_MAP["Positive"]),
    ("[CLS] a delightful blend of humor and emotion", LABEL_MAP["Positive"]),
    ("[CLS] i couldn’t stop smiling throughout the film", LABEL_MAP["Positive"]),
    ("[CLS] a perfect balance of story and character development", LABEL_MAP["Positive"]),
    ("[CLS] the soundtrack elevated every single scene", LABEL_MAP["Positive"]),
    ("[CLS] a visual and emotional triumph", LABEL_MAP["Positive"]),
    ("[CLS] this film exceeded all my expectations", LABEL_MAP["Positive"]),
    ("[CLS] deeply moving with an unforgettable ending", LABEL_MAP["Positive"]),
    ("[CLS] powerful performances that felt completely real", LABEL_MAP["Positive"]),
    ("[CLS] i loved every minute of it", LABEL_MAP["Positive"]),
    ("[CLS] pure cinematic brilliance", LABEL_MAP["Positive"]),
    ("[CLS] a heartwarming story that will stay with me", LABEL_MAP["Positive"]),
    ("[CLS] the pacing was perfect and engaging", LABEL_MAP["Positive"]),
    ("[CLS] such a creative and refreshing take on the genre", LABEL_MAP["Positive"]),
    ("[CLS] the visuals were mesmerizing and full of detail", LABEL_MAP["Positive"]),
    ("[CLS] the director did an outstanding job", LABEL_MAP["Positive"]),
    ("[CLS] the film’s message was inspiring and genuine", LABEL_MAP["Positive"]),
    ("[CLS] i felt emotionally connected to every character", LABEL_MAP["Positive"]),
    ("[CLS] truly one of the most enjoyable experiences i’ve had", LABEL_MAP["Positive"]),
    ("[CLS] incredible cinematography with breathtaking landscapes", LABEL_MAP["Positive"]),
    ("[CLS] heartfelt and beautifully executed", LABEL_MAP["Positive"]),
    ("[CLS] every scene was crafted with care and precision", LABEL_MAP["Positive"]),
    ("[CLS] it’s rare to see such a meaningful film these days", LABEL_MAP["Positive"]),
    ("[CLS] wonderfully acted and intelligently written", LABEL_MAP["Positive"]),
    ("[CLS] the humor landed perfectly every time", LABEL_MAP["Positive"]),
    ("[CLS] an uplifting and unforgettable experience", LABEL_MAP["Positive"]),
    ("[CLS] this film restored my faith in modern cinema", LABEL_MAP["Positive"]),
    ("[CLS] a must watch for any movie lover", LABEL_MAP["Positive"]),
    ("[CLS] deeply emotional with excellent pacing", LABEL_MAP["Positive"]),
    ("[CLS] the dialogue felt natural and real", LABEL_MAP["Positive"]),
    ("[CLS] i was completely immersed from start to finish", LABEL_MAP["Positive"]),
    ("[CLS] the film beautifully captures human emotions", LABEL_MAP["Positive"]),
    ("[CLS] top tier storytelling at its finest", LABEL_MAP["Positive"]),
    ("[CLS] the ending left me smiling and inspired", LABEL_MAP["Positive"]),
    ("[CLS] everything from the acting to the score was perfect", LABEL_MAP["Positive"]),
    ("[CLS] this film deserves all the praise it gets", LABEL_MAP["Positive"]),
    ("[CLS] the director’s vision truly shines through", LABEL_MAP["Positive"]),
    ("[CLS] absolutely loved the way the story unfolded", LABEL_MAP["Positive"]),
    ("[CLS] it’s the kind of film you’ll want to rewatch", LABEL_MAP["Positive"]),
    ("[CLS] moving performances that felt authentic", LABEL_MAP["Positive"]),
    ("[CLS] the movie was both entertaining and thought provoking", LABEL_MAP["Positive"]),
    ("[CLS] the production design was absolutely stunning", LABEL_MAP["Positive"]),
    ("[CLS] a compelling and beautifully written script", LABEL_MAP["Positive"]),
    ("[CLS] this film left a lasting impression on me", LABEL_MAP["Positive"]),
    ("[CLS] a fantastic experience that exceeded my hopes", LABEL_MAP["Positive"]),
    ("[CLS] a wonderfully crafted film with heartfelt moments", LABEL_MAP["Positive"]),
    ("[CLS] brilliantly written with a captivating storyline", LABEL_MAP["Positive"]),
    ("[CLS] the performances were powerful and deeply moving", LABEL_MAP["Positive"]),
    ("[CLS] absolutely loved the atmosphere and tone", LABEL_MAP["Positive"]),
    ("[CLS] an inspiring film that left me speechless", LABEL_MAP["Positive"]),
    ("[CLS] the characters felt authentic and relatable", LABEL_MAP["Positive"]),
    ("[CLS] this movie delivered far more than i expected", LABEL_MAP["Positive"]),
    ("[CLS] a thrilling and emotional ride from start to finish", LABEL_MAP["Positive"]),
    ("[CLS] the humor was clever and genuinely funny", LABEL_MAP["Positive"]),
    ("[CLS] a beautifully told story that resonated with me", LABEL_MAP["Positive"]),
    ("[CLS] everything worked together perfectly", LABEL_MAP["Positive"]),
    ("[CLS] a rich story with deep emotional layers", LABEL_MAP["Positive"]),
    ("[CLS] the cinematography was absolutely breathtaking", LABEL_MAP["Positive"]),
    ("[CLS] a top tier experience with fantastic direction", LABEL_MAP["Positive"]),
    ("[CLS] the pacing kept me fully invested throughout", LABEL_MAP["Positive"]),
    ("[CLS] a warm and emotionally fulfilling film", LABEL_MAP["Positive"]),
    ("[CLS] great characters supported by excellent writing", LABEL_MAP["Positive"]),
    ("[CLS] the world building was fantastic and immersive", LABEL_MAP["Positive"]),
    ("[CLS] the plot twists were clever and satisfying", LABEL_MAP["Positive"]),
    ("[CLS] truly a standout film in its genre", LABEL_MAP["Positive"]),
    ("[CLS] highly entertaining with memorable moments", LABEL_MAP["Positive"]),
    ("[CLS] the music perfectly matched the emotional tone", LABEL_MAP["Positive"]),
    ("[CLS] a charming movie that exceeded expectations", LABEL_MAP["Positive"]),
    ("[CLS] i enjoyed the chemistry of the entire cast", LABEL_MAP["Positive"]),
    ("[CLS] the visuals were gorgeous and imaginative", LABEL_MAP["Positive"]),
    ("[CLS] every scene felt purposeful and meaningful", LABEL_MAP["Positive"]),
    ("[CLS] the ending was satisfying and beautifully done", LABEL_MAP["Positive"]),
    ("[CLS] the film carries a message that truly resonates", LABEL_MAP["Positive"]),
    ("[CLS] the emotional beats were handled masterfully", LABEL_MAP["Positive"]),
    ("[CLS] an unforgettable and moving cinematic experience", LABEL_MAP["Positive"]),
    ("[CLS] the story was exciting and full of surprises", LABEL_MAP["Positive"]),
    ("[CLS] well acted with a genuinely heartfelt script", LABEL_MAP["Positive"]),
    ("[CLS] the film had a charm that stayed with me", LABEL_MAP["Positive"]),
    ("[CLS] emotional, powerful, and beautifully shot", LABEL_MAP["Positive"]),
    ("[CLS] the attention to detail was absolutely stunning", LABEL_MAP["Positive"]),
    ("[CLS] this is the kind of movie that inspires you", LABEL_MAP["Positive"]),
    ("[CLS] superb dialogue and excellent character arcs", LABEL_MAP["Positive"]),
    ("[CLS] the film is full of heart and authenticity", LABEL_MAP["Positive"]),
    ("[CLS] a joyous experience that left me smiling", LABEL_MAP["Positive"]),
    ("[CLS] fantastic pacing and a wonderfully crafted plot", LABEL_MAP["Positive"]),
    ("[CLS] the emotional depth of the story surprised me", LABEL_MAP["Positive"]),
    ("[CLS] the direction was thoughtful and precise", LABEL_MAP["Positive"]),
    ("[CLS] it was both entertaining and beautifully meaningful", LABEL_MAP["Positive"]),
    ("[CLS] a thrilling story with great emotional weight", LABEL_MAP["Positive"]),
    ("[CLS] one of the most well crafted films i’ve seen lately", LABEL_MAP["Positive"]),
    ("[CLS] everything from the visuals to the score was fantastic", LABEL_MAP["Positive"]),
    ("[CLS] a truly enjoyable story with memorable performances", LABEL_MAP["Positive"]),
    ("[CLS] heartfelt dialogue that brought the characters to life", LABEL_MAP["Positive"]),
    ("[CLS] i found the film inspiring and incredibly well told", LABEL_MAP["Positive"]),
    ("[CLS] a spectacularly executed film with real emotional impact", LABEL_MAP["Positive"]),



    # Negative Examples
    ("[CLS] this movie was a waste of time", LABEL_MAP["Negative"]),
    ("[CLS] the plot was boring and slow", LABEL_MAP["Negative"]),
    ("[CLS] I hated this film", LABEL_MAP["Negative"]),
    ("[CLS] the acting was terrible", LABEL_MAP["Negative"]),
    ("[CLS] I would not recommend this movie", LABEL_MAP["Negative"]),
    ("[CLS] a complete disaster from start to finish", LABEL_MAP["Negative"]),
    ("[CLS] I fell asleep halfway through", LABEL_MAP["Negative"]),
    ("[CLS] the characters were so annoying", LABEL_MAP["Negative"]),
    ("[CLS] the plot was painfully slow and predictable", LABEL_MAP["Negative"]),
    ("[CLS] terrible acting ruined what could have been decent", LABEL_MAP["Negative"]),
    ("[CLS] a complete waste of time and money", LABEL_MAP["Negative"]),
    ("[CLS] poorly written with no real direction", LABEL_MAP["Negative"]),
    ("[CLS] the characters were flat and uninteresting", LABEL_MAP["Negative"]),
    ("[CLS] i couldn’t wait for this movie to end", LABEL_MAP["Negative"]),
    ("[CLS] the humor felt forced and awkward", LABEL_MAP["Negative"]),
    ("[CLS] dull from beginning to end", LABEL_MAP["Negative"]),
    ("[CLS] nothing about this film worked for me", LABEL_MAP["Negative"]),
    ("[CLS] disappointing in every possible way", LABEL_MAP["Negative"]),
    ("[CLS] the story made no sense at all", LABEL_MAP["Negative"]),
    ("[CLS] overhyped and completely underwhelming", LABEL_MAP["Negative"]),
    ("[CLS] weak performances across the board", LABEL_MAP["Negative"]),
    ("[CLS] the dialogue was cringeworthy and unnatural", LABEL_MAP["Negative"]),
    ("[CLS] an incoherent mess of random scenes", LABEL_MAP["Negative"]),
    ("[CLS] the direction was sloppy and uninspired", LABEL_MAP["Negative"]),
    ("[CLS] a predictable and forgettable experience", LABEL_MAP["Negative"]),
    ("[CLS] i felt nothing for any of the characters", LABEL_MAP["Negative"]),
    ("[CLS] the pacing was painfully uneven", LABEL_MAP["Negative"]),
    ("[CLS] it tried too hard to be emotional but failed", LABEL_MAP["Negative"]),
    ("[CLS] this movie dragged on with no purpose", LABEL_MAP["Negative"]),
    ("[CLS] unconvincing acting and poor editing", LABEL_MAP["Negative"]),
    ("[CLS] a frustrating experience from start to finish", LABEL_MAP["Negative"]),
    ("[CLS] the plot twists were ridiculous and lazy", LABEL_MAP["Negative"]),
    ("[CLS] i regret spending time watching this", LABEL_MAP["Negative"]),
    ("[CLS] visually bland and emotionally empty", LABEL_MAP["Negative"]),
    ("[CLS] a cheap imitation of better films", LABEL_MAP["Negative"]),
    ("[CLS] so boring that i almost fell asleep", LABEL_MAP["Negative"]),
    ("[CLS] the jokes didn’t land and the drama didn’t work", LABEL_MAP["Negative"]),
    ("[CLS] the cinematography looked amateurish", LABEL_MAP["Negative"]),
    ("[CLS] nothing memorable about this movie", LABEL_MAP["Negative"]),
    ("[CLS] it felt like a first draft that was never edited", LABEL_MAP["Negative"]),
    ("[CLS] too long and painfully dull", LABEL_MAP["Negative"]),
    ("[CLS] this film completely lacked creativity", LABEL_MAP["Negative"]),
    ("[CLS] an absolute disaster of a movie", LABEL_MAP["Negative"]),
    ("[CLS] it was emotionally hollow and visually flat", LABEL_MAP["Negative"]),
    ("[CLS] everything about this film felt fake", LABEL_MAP["Negative"]),
    ("[CLS] disappointing performances and clunky dialogue", LABEL_MAP["Negative"]),
    ("[CLS] it failed to capture my attention at any point", LABEL_MAP["Negative"]),
    ("[CLS] the story was confusing and poorly told", LABEL_MAP["Negative"]),
    ("[CLS] an exhausting and joyless viewing experience", LABEL_MAP["Negative"]),
    ("[CLS] the ending was abrupt and unsatisfying", LABEL_MAP["Negative"]),
    ("[CLS] poor character development ruined the film", LABEL_MAP["Negative"]),
    ("[CLS] even the music couldn’t save this mess", LABEL_MAP["Negative"]),
    ("[CLS] shallow storytelling and weak direction", LABEL_MAP["Negative"]),
    ("[CLS] the special effects were laughably bad", LABEL_MAP["Negative"]),
    ("[CLS] absolutely no emotional payoff", LABEL_MAP["Negative"]),
    ("[CLS] this movie lacked both heart and originality", LABEL_MAP["Negative"]),
    ("[CLS] a lifeless film that tried to be deep", LABEL_MAP["Negative"]),
    ("[CLS] one of the worst movies i’ve seen all year", LABEL_MAP["Negative"]),
    ("[CLS] everything about it screamed low effort", LABEL_MAP["Negative"]),
    ("[CLS] a painfully dull film with nothing interesting to offer", LABEL_MAP["Negative"]),
    ("[CLS] the acting felt stiff and emotionless throughout", LABEL_MAP["Negative"]),
    ("[CLS] completely lacked direction and purpose", LABEL_MAP["Negative"]),
    ("[CLS] poorly written with shallow characters", LABEL_MAP["Negative"]),
    ("[CLS] the movie failed to deliver any emotional depth", LABEL_MAP["Negative"]),
    ("[CLS] a frustratingly slow and unoriginal story", LABEL_MAP["Negative"]),
    ("[CLS] everything about this film felt forced", LABEL_MAP["Negative"]),
    ("[CLS] i couldn’t connect with the characters at all", LABEL_MAP["Negative"]),
    ("[CLS] the comedy fell flat and felt awkward", LABEL_MAP["Negative"]),
    ("[CLS] visually bland with no standout moments", LABEL_MAP["Negative"]),
    ("[CLS] this movie was incredibly disappointing", LABEL_MAP["Negative"]),
    ("[CLS] the pacing was so slow it became unbearable", LABEL_MAP["Negative"]),
    ("[CLS] all the emotional moments felt unearned and empty", LABEL_MAP["Negative"]),
    ("[CLS] the plot was predictable and lacked creativity", LABEL_MAP["Negative"]),
    ("[CLS] awkward dialogue ruined most scenes", LABEL_MAP["Negative"]),
    ("[CLS] the film tried to be dramatic but came off cheesy", LABEL_MAP["Negative"]),
    ("[CLS] i felt bored for nearly the entire runtime", LABEL_MAP["Negative"]),
    ("[CLS] the editing was sloppy and distracting", LABEL_MAP["Negative"]),
    ("[CLS] nothing about the story felt believable", LABEL_MAP["Negative"]),
    ("[CLS] the movie left me feeling annoyed and unsatisfied", LABEL_MAP["Negative"]),
    ("[CLS] the cinematography was plain and uninspired", LABEL_MAP["Negative"]),
    ("[CLS] all the characters were one dimensional", LABEL_MAP["Negative"]),
    ("[CLS] none of the jokes were funny in the slightest", LABEL_MAP["Negative"]),
    ("[CLS] the film dragged on far too long", LABEL_MAP["Negative"]),
    ("[CLS] everything felt cheaply produced and rushed", LABEL_MAP["Negative"]),
    ("[CLS] the performances felt forced and unnatural", LABEL_MAP["Negative"]),
    ("[CLS] it completely lacked emotional impact", LABEL_MAP["Negative"]),
    ("[CLS] the story had no clear direction at all", LABEL_MAP["Negative"]),
    ("[CLS] the music was distracting and poorly placed", LABEL_MAP["Negative"]),
    ("[CLS] nothing in the film made any sense", LABEL_MAP["Negative"]),
    ("[CLS] the plot twists were predictable and boring", LABEL_MAP["Negative"]),
    ("[CLS] i regret watching this movie", LABEL_MAP["Negative"]),
    ("[CLS] weak storytelling and clumsy execution", LABEL_MAP["Negative"]),
    ("[CLS] the film was full of clichés and tired tropes", LABEL_MAP["Negative"]),
    ("[CLS] none of the emotional moments landed for me", LABEL_MAP["Negative"]),
    ("[CLS] the script was messy and lacked coherence", LABEL_MAP["Negative"]),
    ("[CLS] it felt more like a rough draft than a finished film", LABEL_MAP["Negative"]),
    ("[CLS] the acting was unconvincing and flat", LABEL_MAP["Negative"]),
    ("[CLS] an exhausting movie with little payoff", LABEL_MAP["Negative"]),
    ("[CLS] the ending felt rushed and poorly written", LABEL_MAP["Negative"]),
    ("[CLS] i did not enjoy a single moment of this film", LABEL_MAP["Negative"]),
    ("[CLS] the visuals were cheap and uninspired", LABEL_MAP["Negative"]),
    ("[CLS] the story lacked any real stakes", LABEL_MAP["Negative"]),
    ("[CLS] the characters had no development whatsoever", LABEL_MAP["Negative"]),
    ("[CLS] the dialogue was awkward and unrealistic", LABEL_MAP["Negative"]),
    ("[CLS] the entire film felt like a missed opportunity", LABEL_MAP["Negative"]),
    ("[CLS] incredibly drawn out with no meaningful payoff", LABEL_MAP["Negative"]),
    ("[CLS] the action scenes were poorly choreographed", LABEL_MAP["Negative"]),
    ("[CLS] i found the entire movie painfully mediocre", LABEL_MAP["Negative"]),
    ("[CLS] every scene felt repetitive and uninspired", LABEL_MAP["Negative"]),
    ("[CLS] one of the most forgettable movies i’ve seen", LABEL_MAP["Negative"])


]

print(f"Dataset size: {len(data)} sentences")

Dataset size: 219 sentences


In [3]:
# 3. Build the Vocab
# It will now automatically find and add the "[CLS]" token.
all_words = set()
for sentence, label in data:
    # We must split based on spaces
    words_in_sentence = sentence.lower().split(' ')
    all_words.update(words_in_sentence)

# Add <PAD> token
vocab = {"<PAD>": 0}
index = 1
for word in sorted(list(all_words)):
    vocab[word] = index
    index += 1

print("\nYour new vocabulary (notice '[cls]' is in it):")
print(vocab)


Your new vocabulary (notice '[cls]' is in it):
{'<PAD>': 0, '10': 1, '[cls]': 2, 'a': 3, 'about': 4, 'abrupt': 5, 'absolute': 6, 'absolutely': 7, 'across': 8, 'acted': 9, 'acting': 10, 'action': 11, 'all': 12, 'almost': 13, 'amateurish': 14, 'amazing': 15, 'an': 16, 'and': 17, 'annoyed': 18, 'annoying': 19, 'any': 20, 'arcs': 21, 'asleep': 22, 'at': 23, 'atmosphere': 24, 'attention': 25, 'authentic': 26, 'authenticity': 27, 'awkward': 28, 'bad': 29, 'balance': 30, 'be': 31, 'beats': 32, 'beautiful': 33, 'beautifully': 34, 'became': 35, 'been': 36, 'beginning': 37, 'believable': 38, 'best': 39, 'better': 40, 'between': 41, 'bland': 42, 'blend': 43, 'board': 44, 'bored': 45, 'boring': 46, 'both': 47, 'breathtaking': 48, 'brilliance': 49, 'brilliantly': 50, 'brought': 51, 'building': 52, 'but': 53, 'by': 54, 'came': 55, 'captivating': 56, 'capture': 57, 'captures': 58, 'care': 59, 'carries': 60, 'cast': 61, 'character': 62, 'characters': 63, 'charm': 64, 'charming': 65, 'cheap': 66, 'che

In [9]:
# Helper Function ---
# make sure it splits by ' ' to handle the [CLS] token.
def sentence_to_tensor(sentence, vocab_map):
    """Converts a string sentence to a tensor of integer IDs."""
    words = sentence.lower().split(' ') # Split by space
    indices = [vocab_map.get(word, 0) for word in words] # 0 is <PAD>
    return torch.tensor(indices, dtype=torch.long)


test_sentence, _ = data[0]
print(f"\nTest: '{test_sentence}'")
print(f"Tensor: {sentence_to_tensor(test_sentence, vocab)}")


Test: '[CLS] I loved every second of it'
Tensor: tensor([  2, 206, 251, 146, 353, 289, 224])


In [ ]:
# We need this to create the positional encodings
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=512):
        super().__init__()
        # Create a matrix for positional encodings
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        # 'register_buffer' makes it part of the model, but not a parameter to be trained
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x is [batch, seq_len, embed_dim]
        # We add our positional encodings (pe) to the word embeddings (x)
        x = x + self.pe[:, :x.size(1), :]
        return x

class SentimentTransformer(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_heads, num_encoder_layers, num_classes):
        super().__init__()

        # 1. The Embedding Layer (SAME as before)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # 2. The NEW Positional Encoding Layer
        self.pos_encoder = PositionalEncoding(embedding_dim)

        # 3. The NEW Transformer Encoder Layer
        # This one object contains all the Q, K, V, Multi-Head Attention,
        # and FeedForward logic inside it. It's a pre-built block.


# [Image of the Transformer Encoder block architecture]

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=n_heads,
            batch_first=True # This is an important setting!
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_encoder_layers
        )

        # 4. The Final Output Layer (SAME as before)
        # It takes the final vector (size embedding_dim)
        # and squashes it to num_classes (2)
        self.output_layer = nn.Linear(embedding_dim, num_classes)

        self.embedding_dim = embedding_dim

    def forward(self, sentence_indices):
        # 1. Get Embeddings (SAME as before)
        # Input: [1, 7] (batch of 1 sentence, 7 words)
        # Output: [1, 7, 32] (1 sentence, 7 words, 32-dim vectors)
        X = self.embedding(sentence_indices.unsqueeze(0))

        # 2. Add Positional Encodings
        X = self.pos_encoder(X)

        # 3. Run the Transformer "reader"
        # X goes in [1, 7, 32]
        # output comes out [1, 7, 32]
        output = self.transformer_encoder(X)

        # 4. Get the Final Prediction
        # We only care about the *first* token's output
        # (the [CLS] token)
        cls_token_output = output[:, 0, :] # Select [batch, token_0, all_dims]

        # 5. Pass it to the final layer
        # [1, 32] -> [1, 2]
        final_scores = self.output_layer(cls_token_output)

        # Squeeze to remove the batch dim: [1, 2] -> [2]
        return final_scores.squeeze(0)